Начнем с загрузки и предобработки данных

Изначально я работал в colab notebook и для лучшего чтения лучше бы использовать ipynb формат, но просили в py - так что кто я такой чтобы спорить)

In [13]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import re
from tqdm.auto import tqdm
import ast

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# Файл слишком большой, поэтому загружу на диск, чтобы постоянно заново его не читать
file_path = "drive/MyDrive/emailTextsI.csv"


df = pd.read_csv(file_path)

<ipython-input-3-cea2c7f53a11>:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
df

,id,label,text,introduction
0,0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,NaN
1,1,Ham,got ice thought look az original message ice o...,NaN
2,2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,NaN
3,3,Spam,start increasing your odds of success & live s...,NaN
4,4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...
...,...,...,...,...
193847,193847,Ham,on escapenumber escapenumber escapenumber rob ...,NaN
193848,193848,Spam,we have everything you need escapelong cialesc...,NaN
193849,193849,Ham,hi quick question say i have a date variable i...,NaN
193850,193850,Spam,thank you for your loan request which we recie...,NaN


-

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193852 entries, 0 to 193851
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            193852 non-null  int64 
 1   label         193852 non-null  object
 2   text          193850 non-null  object
 3   introduction  19 non-null      object
dtypes: int64(1), object(3)
memory usage: 5.9+ MB


In [ ]:
df[df['text'].isnull()]

,id,label,text,introduction
47294,47294,Spam,NaN,NaN
149214,149214,Ham,NaN,NaN


In [ ]:
# По сути пустой текст в этих двух значениях никакой информации не даст, и восстановить текст никак не получится - поэтому удалим их
df.dropna(subset=['text'], inplace=True)

In [ ]:
# Признак id явно лишний, так как просто дублирует индексацию
df.drop('id', axis=1, inplace=True)

In [ ]:
df

,label,text,introduction
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,NaN
1,Ham,got ice thought look az original message ice o...,NaN
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,NaN
3,Spam,start increasing your odds of success & live s...,NaN
4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...
...,...,...,...
193847,Ham,on escapenumber escapenumber escapenumber rob ...,NaN
193848,Spam,we have everything you need escapelong cialesc...,NaN
193849,Ham,hi quick question say i have a date variable i...,NaN
193850,Spam,thank you for your loan request which we recie...,NaN


In [ ]:
df.duplicated().sum()

0

Дубликатов и пропусков нет, можем приступать к задаче. Чтобы выделить привествия, для начала необходимо определить несколько случаев:


*   Привествие может начинаться с базовых фраз: "Dear", "Hello", "Hi" итд
*   Ссылки на предыдущие сообщения или контекст: "Following up on", "Regarding our conversation" итд
* Автоматически сгенерированные строки: "This message is intended for" итд
* Прочие шаблоны



Гадать можно бесконечно долго, так как датасет немаленький и подбирать все возможные случаи для 190к значений - гемор. Попробуем использовать токенизацию и анализ стоп-слов - думаю такой подход будет достаточно эффективен, учитывая что мы работаем с текстом

In [11]:
# Скачаем необходимый набор стоп-слов и пакет токенизации
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Функция для токенизации и фильтрации стоп-слов
def tokenize_and_filter(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords.words('english')]
    return filtered_tokens

In [ ]:
# Чтобы не портить исходный датасет, пока поработаю в копии
df_copy = df

In [ ]:
df_copy.head()

,label,text,introduction
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,NaN
1,Ham,got ice thought look az original message ice o...,NaN
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,NaN
3,Spam,start increasing your odds of success & live s...,NaN
4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...


In [ ]:
tqdm.pandas(desc="Processing")

# Применение токенизации к первым N словам каждого сообщения
N = 10  # Решил взять как порог 10 - длиннее обращения еще не встречал)
df_copy['first_n_tokens'] = df_copy['text'].progress_apply(lambda x: tokenize_and_filter(x)[:N])

Processing:   0%|          | 0/193850 [00:00<?, ?it/s]

In [ ]:
df_copy

,label,text,introduction,first_n_tokens
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,NaN,"[viiiiiiagraaaa, ones, want, make, scream, ., ..."
1,Ham,got ice thought look az original message ice o...,NaN,"[got, ice, thought, look, az, original, messag..."
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,NaN,"[yo, ur, wom, ne, eds, escapenumber, ch, n, b, e]"
3,Spam,start increasing your odds of success & live s...,NaN,"[start, increasing, odds, success, &, live, se..."
4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...,"[author, jra, date, escapenumber, escapenumber..."
...,...,...,...,...
193847,Ham,on escapenumber escapenumber escapenumber rob ...,NaN,"[escapenumber, escapenumber, escapenumber, rob..."
193848,Spam,we have everything you need escapelong cialesc...,NaN,"[everything, need, escapelong, cialescapenumbe..."
193849,Ham,hi quick question say i have a date variable i...,NaN,"[hi, quick, question, say, date, variable, dat..."
193850,Spam,thank you for your loan request which we recie...,NaN,"[thank, loan, request, recieved, escapenumber,..."


In [ ]:
output_file_path = '/content/drive/MyDrive/processed_emails.csv'
df_copy.to_csv(output_file_path, index=False)

Продолжим работать с сохраненным файлом

In [20]:
file_path = "drive/MyDrive/processed_emails.csv"


df_copy = pd.read_csv(file_path)

<ipython-input-20-317053c6fbdb>:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_copy = pd.read_csv(file_path)


In [6]:
df_copy

,label,text,introduction,first_n_tokens
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,NaN,"['viiiiiiagraaaa', 'ones', 'want', 'make', 'sc..."
1,Ham,got ice thought look az original message ice o...,NaN,"['got', 'ice', 'thought', 'look', 'az', 'origi..."
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,NaN,"['yo', 'ur', 'wom', 'ne', 'eds', 'escapenumber..."
3,Spam,start increasing your odds of success & live s...,NaN,"['start', 'increasing', 'odds', 'success', '&'..."
4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...,"['author', 'jra', 'date', 'escapenumber', 'esc..."
...,...,...,...,...
193845,Ham,on escapenumber escapenumber escapenumber rob ...,NaN,"['escapenumber', 'escapenumber', 'escapenumber..."
193846,Spam,we have everything you need escapelong cialesc...,NaN,"['everything', 'need', 'escapelong', 'cialesca..."
193847,Ham,hi quick question say i have a date variable i...,NaN,"['hi', 'quick', 'question', 'say', 'date', 'va..."
193848,Spam,thank you for your loan request which we recie...,NaN,"['thank', 'loan', 'request', 'recieved', 'esca..."


Теперь мы можем выполнить частотный анализ этих слов, чтобы определить наиболее распространенные вступительные фразы

In [21]:
df_copy['first_n_tokens'] = df_copy['first_n_tokens'].apply(ast.literal_eval)

all_tokens = [token for sublist in df_copy['first_n_tokens'] for token in sublist]
token_counter = Counter(all_tokens)
most_common_tokens = token_counter.most_common(100)

In [22]:
most_common_tokens

[('escapenumber', 134455),
 ('-', 16160),
 ('.', 16148),
 (',', 14364),
 ('message', 13880),
 ('original', 11674),
 ('escapelong', 10132),
 ('com', 10118),
 ('please', 9562),
 ('dear', 9122),
 ('http', 8957),
 ('sent', 8937),
 ('new', 7270),
 ('hi', 7207),
 ('enron', 7067),
 ('get', 7020),
 ("'s", 6549),
 ('software', 6150),
 ('attached', 5943),
 (':', 5874),
 ('e', 5433),
 ('one', 5302),
 ("'", 5254),
 ('wrote', 5136),
 ('hello', 5049),
 ('would', 4963),
 ('like', 4809),
 ('need', 4762),
 ('know', 4638),
 ('see', 4587),
 ('/', 4564),
 ('thanks', 4496),
 ('may', 4490),
 ('date', 4484),
 ('email', 4271),
 ('day', 4188),
 ('time', 3927),
 ('want', 3920),
 ('b', 3850),
 ('subject', 3806),
 ('r', 3580),
 ('www', 3473),
 ('us', 3416),
 ('today', 3405),
 ("n't", 3311),
 ('good', 3186),
 ('following', 3122),
 ('?', 3087),
 ('!', 3073),
 ('online', 3068),
 ('call', 3028),
 ('find', 2980),
 ('price', 2922),
 ('let', 2904),
 ('meeting', 2902),
 ('prices', 2883),
 ('monday', 2814),
 ('p', 2787),


Теперь у нас есть список наиболее часто встречающихся слов в первых N токенах каждого сообщения. Этот список может помочь определить шаблонные фразы, которые часто встречаются в начале эмейлов


Например, слова вроде "dear", "hi", "hello" могут быть частью приветствия, а "please", "thanks" и "attached" могут указывать на стандартные обороты в деловой переписке. Однако, есть и другие слова как "escapenumber" и подобные ему ("escapelong", "com", "http") могут быть частью автоматически сгенерированных шаблонов или системных сообщений

Теперь воспользуемся регулярками для извлечения вступительных фраз

In [57]:
intro_patterns = [
    r"\bDear\s+\w+",  # Приветствие с 'Dear'
    r"\bHello\s+\w+",  # Приветствие с 'Hello'
    r"\bHi\b",  # Приветствие с 'Hi'
    r"\bPlease\s+\w+",  # Вежливое начало с 'Please'
    r"\bThank\s+you",  # Благодарность в начале с 'Thank you'
    r"\bThanks\b",  # Благодарность в начале с 'Thanks'
    r"\bHey\s+\w+",  # Приветствие с 'Hey'
    r"\bMessage\b",  # Начало с пересылаемого сообщения
    r"\bAttached\b",  # Упоминание прикрепленного файла с 'Attached'
    r"\bOriginal\s+\w+",  # Начало с 'Original'
    r"\bSent\b",  # Начало с 'Sent'
    r"\bWrote\s+\w+",  # Начало с ответа на сообщение
]

In [78]:
# Функция для извлечения вступительной части сообщения
def extract_introduction(text, patterns, num_words_check=10):
    words = text.split()[:num_words_check]
    text_to_search = ' '.join(words)

    for pattern in patterns:
        match = re.search(pattern, text_to_search, re.IGNORECASE)  # Используем re.IGNORECASE для поиска без учета регистра
        if match:
            return text_to_search[:match.end()].strip()
    return None

In [79]:
tqdm.pandas(desc="Processing")

df_copy['introduction'] = df_copy['text'].progress_apply(lambda x: extract_introduction(x, intro_patterns))

Processing:   0%|          | 0/193850 [00:00<?, ?it/s]

In [80]:
df_copy

,label,text,introduction,first_n_tokens
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,None,"[viiiiiiagraaaa, ones, want, make, scream, ., ..."
1,Ham,got ice thought look az original message ice o...,got ice thought look az original message,"[got, ice, thought, look, az, original, messag..."
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,None,"[yo, ur, wom, ne, eds, escapenumber, ch, n, b, e]"
3,Spam,start increasing your odds of success & live s...,None,"[start, increasing, odds, success, &, live, se..."
4,Ham,author jra date escapenumber escapenumber esca...,None,"[author, jra, date, escapenumber, escapenumber..."
...,...,...,...,...
193845,Ham,on escapenumber escapenumber escapenumber rob ...,on escapenumber escapenumber escapenumber rob ...,"[escapenumber, escapenumber, escapenumber, rob..."
193846,Spam,we have everything you need escapelong cialesc...,None,"[everything, need, escapelong, cialescapenumbe..."
193847,Ham,hi quick question say i have a date variable i...,hi,"[hi, quick, question, say, date, variable, dat..."
193848,Spam,thank you for your loan request which we recie...,thank you,"[thank, loan, request, recieved, escapenumber,..."


In [82]:
df_copy[df_copy['introduction'].notna()]

,label,text,introduction,first_n_tokens
1,Ham,got ice thought look az original message ice o...,got ice thought look az original message,"[got, ice, thought, look, az, original, messag..."
6,Ham,attached is the weekly deal report from 10 / 1...,attached,"[attached, weekly, deal, report, 10, /, 18, /,..."
10,Ham,hey there - - life sounds horribly busy . i fi...,hey there,"[hey, -, -, life, sounds, horribly, busy, ., f..."
16,Spam,dear customer it's not a secret that there are...,dear customer,"[dear, customer, 's, secret, lots, online, pha..."
25,Ham,hi i am a graduate student at stanford univers...,hi,"[hi, graduate, student, stanford, university, ..."
...,...,...,...,...
193842,Ham,original message from ntops insync net mailto...,original message,"[original, message, ntops, insync, net, mailto..."
193845,Ham,on escapenumber escapenumber escapenumber rob ...,on escapenumber escapenumber escapenumber rob ...,"[escapenumber, escapenumber, escapenumber, rob..."
193847,Ham,hi quick question say i have a date variable i...,hi,"[hi, quick, question, say, date, variable, dat..."
193848,Spam,thank you for your loan request which we recie...,thank you,"[thank, loan, request, recieved, escapenumber,..."


В целом, результат получился неплохой - однако все равно существует ряд неточностей. Какие-то письма были не на английском, например, а какие-то считаются слэнгом и прочими уникальными фразами

In [84]:
# Сохраняем обновленный датасет в файл CSV
output_file_path = '/content/drive/MyDrive/res_emails.csv'
df_copy.to_csv(output_file_path, index=False)